In [ ]:
# TODO: Fill out annotations

In [120]:
import Pkg
Pkg.activate("..")

# these lines are required only for local installations
Pkg.develop(path="../../scp_traj_opt/")
Pkg.precompile()

using SCPToolbox
using ECOS
using SCS
using JuMP
using Printf

  Activating project at `~/ACL/AAS/SCPToolbox_tutorial`
   Resolving package versions...
  No Changes to `~/ACL/AAS/SCPToolbox_tutorial/Project.toml`
  No Changes to `~/ACL/AAS/SCPToolbox_tutorial/Manifest.toml`


In [121]:
# ..:: Construct Subproblem ::..
function construct_subpbm(params)
    
    # Construct the subproblem
    pbm = ConicProgram(params;solver = params["solver"],solver_options = params["opts"])
    
    # Create variables
    z = @new_variable(pbm, 2, "z")
    ν = @new_variable(pbm, "ν")
    η = @new_variable(pbm,"η")
    μ = @new_variable(pbm,"μ")  
    
    ### (Point out usage of dictionary pars inside constraint constructor)
    
    # Linearized nonconvex constraint
    @add_constraint(
        pbm, ZERO, "my-ncvx", (z,ν),
        begin
            local z,ν = arg
            cst["A1"]*z .+ cst["b1"] .+ ν
        end)
    
    # Hyperplane constraint
    @add_constraint(
        pbm, ZERO, "my-hypln", (z,),
        begin
            local z, = arg
            cst["A2"]*z .+ cst["b2"]
        end)
    
    # Halfspace constraint
    @add_constraint(
        pbm, NONPOS, "my-hlfspace", (z,),
        begin
            local z, = arg
            cst["A3"]*z .+ cst["b3"]
        end)
        
    # Trust-region penalty constraint
    @add_constraint(
       pbm, SOC, "my-soc", (z,η),
        begin
            local z,η = arg
            vcat(η,z .- cst["z̄"])
        end
    )
    
    # Virtual buffer penalty constraint
    @add_constraint(
       pbm, SOC, "my-soc2", (ν,μ),
        begin
            local ν,μ = arg
            vcat(μ,ν)
        end
    )
    
    
    # Cost function
    @add_cost(pbm, (η,μ,z),
        begin
            local η,μ,z = arg
            z[2] .+ cst["wtr"]*η .+ cst["wvb"]*μ
        end
    )
    
    return pbm,z,η,μ
    
end

;

In [126]:
# ..:: Initialize ::..

# Construct problem parameters
params = Dict()
params["solver"] = SCS
# params["opts"]   = Dict("verbose"=>0,"abstol"=>1e-8)
# params["opts"] = Dict("print_level"=>0)
params["opts"] = Dict("verbose"=>0)

# Initial reference and constraint data
params["z̄"] = zeros(2);
params["A1"] = Matrix(transpose([-2*params["z̄"][1];1.0]))
params["b1"] = params["z̄"][1]^2
params["A2"] = Matrix(transpose([0.1;1]))
params["b2"] = -0.6
params["A3"] = Matrix(transpose([1.0;0.0]))
params["b3"] = -0.85

iter_max = 30; iter_conv = copy(iter_max)
ϵ_conv = 1e-4
params["wtr"] = 10.0
params["wvb"] = 10000.0
z_hist = [zeros(2) for k = 1:iter_max]
μ_hist = zeros(iter_max)
η_hist = zeros(iter_max)
cost_hist = zeros(iter_max)
;

In [127]:

# ..:: Iterate ::..

k = 1 # Iteration counter
while true
    
    # Construct subproblem
    pbm,z,η,μ = construct_subpbm(params)
    
    # Solve subproblem
    exit_flag = solve!(pbm)
   
    # Save history of the subproblem
    z_hist[k] .= value(z)    
    η_hist[k] = value(η)[1]
    μ_hist[k] = value(μ)[1]
    cost_hist[k] = z_hist[k][2] 

    # Check stopping criterion
    bool_conv = max(η_hist[k],μ_hist[k]) <= ϵ_conv
    
    @printf("Iteration %d | %s | Trust region %.3e | Virtual control %.3e | Cost: %.3f |\n",k,string(exit_flag),η_hist[k],μ_hist[k],cost_hist[k])
        
    # Stop at maximum iterations
    if (k == iter_max) || bool_conv
        iter_conv = k        
        break
    else
    
        # Update next reference trajectory as solution
        params["z̄"] = z_hist[k]

        # Create linearized constraint values
        params["A1"] = Matrix(transpose([-2*params["z̄"][1];1.0]))
        params["b1"] = params["z̄"][1]^2    
        
        k += 1
    end
end


Iteration 1 | OPTIMAL | Trust region 9.938e-01 | Virtual control 5.150e-01 | Cost: 0.515 |
Iteration 2 | OPTIMAL | Trust region 1.159e-01 | Virtual control 5.900e-11 | Cost: 0.527 |
Iteration 3 | OPTIMAL | Trust region 8.510e-03 | Virtual control -9.770e-11 | Cost: 0.527 |
Iteration 4 | OPTIMAL | Trust region 4.641e-05 | Virtual control -5.027e-14 | Cost: 0.527 |


In [128]:
@show z_hist[iter_conv]

z_hist[iter_conv] = [0.726208736186539, 0.5273791263813722]


2-element Vector{Float64}:
 0.726208736186539
 0.5273791263813722

In [ ]:
# TODO: Fill out JuMP code

model = Model(ECOS.Optimizer)
@variable(model,z[1:2])
@const
